# constellation detection Model (BASE MODEL)
Yolov8s
Adam

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8s.pt"  # Using the small variant (11.2M parameters)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv8 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8s model using the dataset
    """
    # Load a pretrained YOLOv8s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",
        lr0=LEARNING_RATE,  # Initial learning rate
        conf=0.25,  # Detection confidence threshold
        name="constellation_detector",
        device=0, 
        amp=False  # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    # Validate the model
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv8s model...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolov8s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Testing AdamW Optimizer
Yolov8s
AdamW

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8s.pt"  # Using the small variant (11.2M parameters)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv8 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8s model using the dataset with AdamW optimizer
    """
    # Load a pretrained YOLOv8s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model with AdamW
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="AdamW",  # Changed from "Adam" to "AdamW"
        lr0=0.001,          # Recommended learning rate for AdamW
        weight_decay=0.01,  # Add weight decay parameter for AdamW
        conf=0.25,          # Detection confidence threshold
        name="constellation_detector_adamw",
        device=0,           
        amp=False           # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv8s model...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolov8s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Testing SGD Optimizer
Yolov8s
SGD

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8s.pt"  # Using the small variant (11.2M parameters)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv8 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8s model using the dataset with SGD+momentum
    """
    # Load a pretrained YOLOv8s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model with SGD+momentum
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="SGD",    # Changed from "Adam" to "SGD"
        lr0=0.01,           # Higher learning rate typically used for SGD
        momentum=0.937,     # Add momentum parameter (standard YOLOv8 value)
        weight_decay=0.0005, # Standard weight decay for SGD
        conf=0.25,          # Detection confidence threshold
        name="constellation_detector_sgd",
        device=0,          
        amp=False           # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv8s model...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolov8s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Testing YOLOv11n
YOLOv11n
Adam

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolo11n.pt"  # Changed from yolov8s.pt to yolov11n.pt (nano variant)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv11 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv11 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv11n model using the dataset
    """
    # Load a pretrained YOLOv11n model
    model = YOLO(MODEL_TYPE)
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam", 
        lr0=LEARNING_RATE,  # Initial learning rate
        conf=0.25,  # Detection confidence threshold
        name="constellation_detector",
        device=0,  
        amp=False,  # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")

    yaml_path = create_yaml_config()
    
    print("Training YOLOv11n model...") 
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolov11n.pt"  # Updated filename
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Testing YOLOv8n
YOLOv8n
Adam

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8n.pt"  # Changed to YOLOv8n (nano variant)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv8 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8n model using the dataset
    """
    # Load a pretrained YOLOv8n model
    model = YOLO(MODEL_TYPE)
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",  
        lr0=LEARNING_RATE,  # Initial learning rate
        conf=0.25,  # Detection confidence threshold
        name="constellation_detector",
        device=0, 
        amp=False  # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"• mAP50: {metrics['mAP50']:.3f}")
    print(f"• Precision: {metrics['precision']:.3f}")
    print(f"• Recall: {metrics['recall']:.3f}")
    print(f"• F1 Score: {metrics['f1']:.3f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv8n model...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolov8n.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Testing YOLOv11s
YOLOv11s
Adam

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolo11s.pt"  # Changed to YOLOv11s (small variant)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv11 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Increase contrast to make stars more visible
        img_enhanced = enhance_star_visibility(img_resized)
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_enhanced)
    
    return

def enhance_star_visibility(image):
    """
    Apply image processing to enhance star visibility
    """
    # Convert to grayscale for brightness analysis
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    
    # If original image was color, convert back to color
    if len(image.shape) == 3:
        # Create a 3-channel image from the equalized grayscale
        equalized_color = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
        return equalized_color
    
    return equalized

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv11 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv11s model using the dataset
    """
    # Load a pretrained YOLOv11s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model with optimized hyperparameters for star detection
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",  
        lr0=LEARNING_RATE,  # Initial learning rate
        lrf=0.01,          # Final learning rate as a fraction of lr0
        momentum=0.937,    # SGD momentum/Adam beta1
        weight_decay=0.0005,  # Optimizer weight decay
        warmup_epochs=3.0,    # Warmup epochs for learning rate
        warmup_momentum=0.8,  # Warmup initial momentum
        box=7.5,           # Box loss gain (higher for better bounding box precision)
        cls=0.5,           # Class loss gain (lower for astronomical objects with less class variation)
        hsv_h=0.015,       # Image HSV-Hue augmentation (fraction)
        hsv_s=0.7,         # Image HSV-Saturation augmentation (fraction)
        hsv_v=0.4,         # Image HSV-Value augmentation (fraction) - important for brightness variations
        degrees=0.0,       # Image rotation (+/- deg) - minimal rotation for astronomical images
        translate=0.1,     # Image translation (+/- fraction)
        scale=0.5,         # Image scale (+/- gain) - helps with detection at different scales
        fliplr=0.5,        # Image flip left-right (probability)
        mosaic=1.0,        # Mosaic augmentation (probability)
        mixup=0.0,         # Mixup augmentation (probability) - disabled as it might confuse constellation patterns
        copy_paste=0.0,    # Segment copy-paste (probability) - not needed for star detection
        conf=0.25,         # Detection confidence threshold
        iou=0.7,           # Intersection over union (IoU) threshold for NMS
        max_det=300,       # Maximum detections per image
        name="constellation_detector",
        device=0,          
        amp=False          # Disable Automatic Mixed Precision to avoid CUDA errors
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"• mAP50: {metrics['mAP50']:.3f}")
    print(f"• Precision: {metrics['precision']:.3f}")
    print(f"• Recall: {metrics['recall']:.3f}")
    print(f"• F1 Score: {metrics['f1']:.3f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image with optimized inference parameters
    """
    # Run inference with parameters optimized for star detection
    results = model.predict(
        image_path, 
        conf=0.25,       # Detection confidence threshold
        iou=0.7,         # NMS IoU threshold
        max_det=300,     # Maximum number of detections per image
        augment=True     # TTA (Test Time Augmentation)
    )
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv11s model with optimized parameters for star constellation detection...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolo11s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")

    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Combining the most optimized optimizer and model architecture
YOLOv11s
AdamW

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001  
MODEL_TYPE = "yolo11s.pt"  # Changed to YOLOv11s (small variant)

CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv11 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        # Increase contrast to make stars more visible
        img_enhanced = enhance_star_visibility(img_resized)
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_enhanced)
    
    return

def enhance_star_visibility(image):
    """
    Apply image processing to enhance star visibility
    """
    # Convert to grayscale for brightness analysis
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    
    # If original image was color, convert back to color
    if len(image.shape) == 3:
        # Create a 3-channel image from the equalized grayscale
        equalized_color = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
        return equalized_color
    
    return equalized

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv11 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv11s model using the dataset with AdamW optimizer
    """
    # Load a pretrained YOLOv11s model
    model = YOLO(MODEL_TYPE)
    
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="AdamW",  # Changed to AdamW optimizer
        lr0=LEARNING_RATE,  # Initial learning rate set to 0.001
        lrf=0.01,          # Final learning rate as a fraction of lr0
        momentum=0.937,    # SGD momentum/Adam beta1 (not used with AdamW)
        weight_decay=0.01,  # AdamW weight decay - increased for better regularization
        warmup_epochs=3.0,    # Warmup epochs for learning rate
        warmup_momentum=0.8,  # Warmup initial momentum
        box=7.5,           # Box loss gain (higher for better bounding box precision)
        cls=0.5,           # Class loss gain (lower for astronomical objects with less class variation)
        hsv_h=0.015,       # Image HSV-Hue augmentation (fraction)
        hsv_s=0.7,         # Image HSV-Saturation augmentation (fraction)
        hsv_v=0.4,         # Image HSV-Value augmentation (fraction) - important for brightness variations
        degrees=0.0,       # Image rotation (+/- deg) - minimal rotation for astronomical images
        translate=0.1,     # Image translation (+/- fraction)
        scale=0.5,         # Image scale (+/- gain) - helps with detection at different scales
        fliplr=0.5,        # Image flip left-right (probability)
        mosaic=1.0,        # Mosaic augmentation (probability)
        mixup=0.0,         # Mixup augmentation (probability) - disabled as it might confuse constellation patterns
        copy_paste=0.0,    # Segment copy-paste (probability) - not needed for star detection
        conf=0.25,         # Detection confidence threshold
        iou=0.7,           # Intersection over union (IoU) threshold for NMS
        max_det=300,       # Maximum detections per image
        name="constellation_detector_adamw",  # Updated name to reflect AdamW usage
        device=0,         
        amp=True           # Enable Automatic Mixed Precision for better performance with AdamW
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"• mAP50: {metrics['mAP50']:.3f}")
    print(f"• Precision: {metrics['precision']:.3f}")
    print(f"• Recall: {metrics['recall']:.3f}")
    print(f"• F1 Score: {metrics['f1']:.3f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image with optimized inference parameters
    """
    # Run inference with parameters optimized for star detection
    results = model.predict(
        image_path, 
        conf=0.25,       # Detection confidence threshold
        iou=0.7,         # NMS IoU threshold
        max_det=300,     # Maximum number of detections per image
        augment=True     # TTA (Test Time Augmentation)
    )
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv11s model with AdamW optimizer (lr=0.001) for star constellation detection...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    model_save_path = "models/constellation_detector_yolo11s_adamw.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

# Adding post processing to optimized model
YOLOv11s
AdamW

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform
from collections import defaultdict
import math

# Configuration
DATASET_PATH = "stars_constellations_dataset"
IMG_SIZE = 640
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001
MODEL_TYPE = "yolo11s.pt"

# Define constellation classes
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Ursa Minor"
]

# Constellation patterns (simplified star patterns for each constellation)
CONSTELLATION_PATTERNS = {
    "Orion": {
        "min_stars": 7,
        "belt_stars": 3,
        "characteristic_shape": "trapezoid_with_belt"
    },
    "Cassiopeia": {
        "min_stars": 5,
        "characteristic_shape": "w_shape"
    },
    "Ursa Major": {
        "min_stars": 7,
        "characteristic_shape": "big_dipper"
    },
    "Pleiades": {
        "min_stars": 6,
        "cluster_density": "high",
        "tight_grouping": True
    }
}

class ConstellationPostProcessor:
    """
    Post-processing class for refining YOLO constellation detections
    """
    
    def __init__(self, confidence_threshold=0.25, nms_threshold=0.7):
        self.confidence_threshold = confidence_threshold
        self.nms_threshold = nms_threshold
        self.constellation_patterns = CONSTELLATION_PATTERNS
        
    def apply_post_processing(self, results, image_shape):
        """
        Apply comprehensive post-processing to YOLO results
        """
        processed_results = []
        
        for result in results:
            # Extract detections
            boxes = result.boxes.xyxy.cpu().numpy()
            scores = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            
            # Step 1: Filter by confidence
            mask = scores >= self.confidence_threshold
            boxes = boxes[mask]
            scores = scores[mask]
            classes = classes[mask]
            
            # Step 2: Apply constellation-specific NMS
            refined_detections = self._apply_constellation_nms(boxes, scores, classes)
            
            # Step 3: Validate constellation patterns
            validated_detections = self._validate_constellation_patterns(refined_detections, image_shape)
            
            # Step 4: Remove impossible overlaps
            final_detections = self._remove_impossible_overlaps(validated_detections)
            
            # Step 5: Apply spatial consistency checks
            consistent_detections = self._apply_spatial_consistency(final_detections, image_shape)
            
            processed_results.append(consistent_detections)
        
        return processed_results
    
    def _apply_constellation_nms(self, boxes, scores, classes):
        """
        Apply Non-Maximum Suppression with constellation-specific rules
        """
        detections = []
        
        # Group detections by class
        class_detections = defaultdict(list)
        for i, cls in enumerate(classes):
            class_detections[cls].append({
                'box': boxes[i],
                'score': scores[i],
                'class': cls
            })
        
        # Apply NMS per class with different thresholds
        for cls, dets in class_detections.items():
            if not dets:
                continue
                
            # Special handling for certain constellations
            constellation_name = CLASSES[cls]
            
            if constellation_name == "Pleiades":
                # More lenient NMS for star clusters
                nms_thresh = 0.3
            elif constellation_name in ["Orion", "Cassiopeia", "Ursa Major"]:
                # Stricter NMS for well-defined patterns
                nms_thresh = 0.7
            else:
                nms_thresh = self.nms_threshold
            
            # Sort by confidence
            dets.sort(key=lambda x: x['score'], reverse=True)
            
            # Apply NMS
            keep = []
            while dets:
                current = dets.pop(0)
                keep.append(current)
                
                # Remove overlapping detections
                remaining = []
                for det in dets:
                    iou = self._calculate_iou(current['box'], det['box'])
                    if iou <= nms_thresh:
                        remaining.append(det)
                dets = remaining
            
            detections.extend(keep)
        
        return detections
    
    def _validate_constellation_patterns(self, detections, image_shape):
        """
        Validate detections against known constellation patterns
        """
        validated = []
        
        # Group detections by constellation
        constellation_groups = defaultdict(list)
        for det in detections:
            constellation_name = CLASSES[det['class']]
            constellation_groups[constellation_name].append(det)
        
        for constellation_name, group in constellation_groups.items():
            if constellation_name in self.constellation_patterns:
                pattern = self.constellation_patterns[constellation_name]
                
                # Check minimum star count
                min_stars = pattern.get('min_stars', 1)
                if len(group) < min_stars:
                    # Keep the highest confidence detection
                    group.sort(key=lambda x: x['score'], reverse=True)
                    validated.extend(group[:1])
                else:
                    # Validate pattern structure
                    if self._validate_spatial_pattern(group, constellation_name):
                        validated.extend(group)
                    else:
                        # Keep top detections if pattern doesn't match perfectly
                        group.sort(key=lambda x: x['score'], reverse=True)
                        validated.extend(group[:min_stars])
            else:
                validated.extend(group)
        
        return validated
    
    def _validate_spatial_pattern(self, detections, constellation_name):
        """
        Validate the spatial arrangement of detected stars
        """
        if len(detections) < 2:
            return True
        
        # Get center points
        centers = []
        for det in detections:
            box = det['box']
            center_x = (box[0] + box[2]) / 2
            center_y = (box[1] + box[3]) / 2
            centers.append([center_x, center_y])
        
        centers = np.array(centers)
        
        # Constellation-specific pattern validation
        if constellation_name == "Cassiopeia":
            return self._validate_w_shape(centers)
        elif constellation_name == "Orion":
            return self._validate_orion_pattern(centers)
        elif constellation_name == "Pleiades":
            return self._validate_cluster_pattern(centers)
        
        return True
    
    def _validate_w_shape(self, centers):
        """
        Validate W-shape pattern for Cassiopeia
        """
        if len(centers) < 5:
            return False
        
        # Sort by x-coordinate
        sorted_centers = centers[centers[:, 0].argsort()]
        
        # Check for alternating y-coordinates (W pattern)
        y_coords = sorted_centers[:, 1]
        if len(y_coords) >= 5:
            # Simple W-shape check: middle point should be lower/higher than neighbors
            middle_idx = len(y_coords) // 2
            if middle_idx > 0 and middle_idx < len(y_coords) - 1:
                return abs(y_coords[middle_idx] - y_coords[middle_idx-1]) > 10 and \
                       abs(y_coords[middle_idx] - y_coords[middle_idx+1]) > 10
        
        return True
    
    def _validate_orion_pattern(self, centers):
        """
        Validate Orion constellation pattern
        """
        if len(centers) < 7:
            return False
        
        # Look for belt stars (3 stars in a line)
        distances = pdist(centers)
        dist_matrix = squareform(distances)
        
        # Find potential belt stars (relatively close to each other)
        belt_candidates = []
        for i in range(len(centers)):
            close_stars = np.where(dist_matrix[i] < 100)[0]  # Within 100 pixels
            if len(close_stars) >= 3:
                belt_candidates.extend(close_stars)
        
        return len(set(belt_candidates)) >= 3
    
    def _validate_cluster_pattern(self, centers):
        """
        Validate tight clustering pattern for Pleiades
        """
        if len(centers) < 6:
            return False
        
        # Calculate average distance between stars
        distances = pdist(centers)
        avg_distance = np.mean(distances)
        std_distance = np.std(distances)
        
        # Pleiades should have relatively uniform, close spacing
        return std_distance / avg_distance < 0.5  # Low variance in distances
    
    def _remove_impossible_overlaps(self, detections):
        """
        Remove detections that overlap impossibly (same constellation can't appear twice)
        """
        # Group by constellation type
        constellation_groups = defaultdict(list)
        for det in detections:
            constellation_name = CLASSES[det['class']]
            constellation_groups[constellation_name].append(det)
        
        filtered = []
        for constellation_name, group in constellation_groups.items():
            if len(group) > 1:
                # For most constellations, keep only the highest confidence detection
                # Exception: Pleiades (can have multiple clusters)
                if constellation_name != "Pleiades":
                    group.sort(key=lambda x: x['score'], reverse=True)
                    filtered.append(group[0])
                else:
                    # For Pleiades, keep non-overlapping clusters
                    filtered.extend(self._filter_overlapping_clusters(group))
            else:
                filtered.extend(group)
        
        return filtered
    
    def _filter_overlapping_clusters(self, detections):
        """
        Filter overlapping Pleiades clusters
        """
        detections.sort(key=lambda x: x['score'], reverse=True)
        filtered = []
        
        for det in detections:
            overlaps = False
            for existing in filtered:
                iou = self._calculate_iou(det['box'], existing['box'])
                if iou > 0.3:  # 30% overlap threshold
                    overlaps = True
                    break
            
            if not overlaps:
                filtered.append(det)
        
        return filtered
    
    def _apply_spatial_consistency(self, detections, image_shape):
        """
        Apply spatial consistency checks based on astronomical knowledge
        """
        filtered = []
        
        for det in detections:
            constellation_name = CLASSES[det['class']]
            box = det['box']
            
            # Check if constellation is in reasonable position
            center_x = (box[0] + box[2]) / 2
            center_y = (box[1] + box[3]) / 2
            
            # Normalize coordinates
            norm_x = center_x / image_shape[1]
            norm_y = center_y / image_shape[0]
            
            # Apply constellation-specific spatial rules
            if constellation_name == "Moon":
                # Moon should be relatively large and circular
                width = box[2] - box[0]
                height = box[3] - box[1]
                aspect_ratio = width / height
                if 0.8 <= aspect_ratio <= 1.2:  # Roughly circular
                    filtered.append(det)
            else:
                # For star constellations, no specific spatial constraints
                filtered.append(det)
        
        return filtered
    
    def _calculate_iou(self, box1, box2):
        """
        Calculate Intersection over Union (IoU) between two bounding boxes
        """
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def visualize_post_processed_results(self, original_results, processed_results, image):
        """
        Visualize comparison between original and post-processed results
        """
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        # Original results
        ax1.imshow(image)
        ax1.set_title("Original YOLO Detections")
        self._draw_detections(ax1, original_results[0] if original_results else [])
        
        # Post-processed results
        ax2.imshow(image)
        ax2.set_title("Post-Processed Detections")
        self._draw_detections(ax2, processed_results[0] if processed_results else [])
        
        plt.tight_layout()
        plt.show()
    
    def _draw_detections(self, ax, detections):
        """
        Draw bounding boxes on the image
        """
        colors = plt.cm.tab20(np.linspace(0, 1, len(CLASSES)))
        
        for det in detections:
            box = det['box']
            score = det['score']
            cls = det['class']
            
            # Draw bounding box
            rect = plt.Rectangle(
                (box[0], box[1]), 
                box[2] - box[0], 
                box[3] - box[1],
                fill=False, 
                color=colors[cls], 
                linewidth=2
            )
            ax.add_patch(rect)
            
            # Add label
            label = f"{CLASSES[cls]}: {score:.2f}"
            ax.text(
                box[0], 
                box[1] - 5, 
                label, 
                fontsize=8, 
                color=colors[cls],
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7)
            )

# Modified detection function with post-processing
def detect_constellations_with_postprocessing(model, image_path, post_processor=None):
    """
    Perform constellation detection with optional post-processing
    """
    # Run inference
    results = model.predict(
        image_path, 
        conf=0.1,        # Lower confidence for post-processing
        iou=0.5,         # Lower IoU for post-processing
        max_det=300,
        augment=True
    )
    
    # Load image for visualization
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_shape = image.shape
    
    if post_processor is not None:
        # Apply post-processing
        processed_results = post_processor.apply_post_processing(results, image_shape)
        
        # Visualize comparison
        post_processor.visualize_post_processed_results(results, processed_results, image)
        
        # Print statistics
        original_count = len(results[0].boxes) if results and results[0].boxes is not None else 0
        processed_count = len(processed_results[0]) if processed_results else 0
        
        print(f"\nPost-processing Statistics:")
        print(f"Original detections: {original_count}")
        print(f"Post-processed detections: {processed_count}")
        print(f"Filtered out: {original_count - processed_count}")
        
        # Print processed detections
        print("\nFinal Detections:")
        for det in processed_results[0]:
            class_name = CLASSES[det['class']]
            confidence = det['score']
            print(f"• {class_name}: {confidence:.3f}")
        
        return processed_results
    else:
        # Display original results
        plt.figure(figsize=(10, 10))
        plt.imshow(results[0].plot())
        plt.axis('off')
        plt.title('Original Constellation Detection')
        plt.show()
        
        return results

def create_dataset_structure():
    """Create the necessary directory structure for YOLOv11 format"""
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)

def preprocess_images(input_dir, output_dir):
    """Preprocess images by resizing to 640x640"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_enhanced = enhance_star_visibility(img_resized)
        
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_enhanced)

def enhance_star_visibility(image):
    """Apply image processing to enhance star visibility"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    equalized = cv2.equalizeHist(gray)
    
    if len(image.shape) == 3:
        equalized_color = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
        return equalized_color
    
    return equalized

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """Split dataset into train, validation and test sets"""
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """Create YAML configuration file for YOLOv11 training"""
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """Train the YOLOv11s model using the dataset with AdamW optimizer"""
    model = YOLO(MODEL_TYPE)
    
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="AdamW",
        lr0=LEARNING_RATE,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.01,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        box=7.5,
        cls=0.5,
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=0.0,
        translate=0.1,
        scale=0.5,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.0,
        copy_paste=0.0,
        conf=0.25,
        iou=0.7,
        max_det=300,
        name="constellation_detector_adamw_postprocessed",
        device=0,
        amp=True
    )
    
    return model, results

def evaluate_model(model):
    """Evaluate model performance using mAP50 and other metrics"""
    results = model.val()
    
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:
                return float(value[0])
            else:
                return float(value)
        except (TypeError, ValueError, IndexError):
            return 0.0
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"• mAP50: {metrics['mAP50']:.3f}")
    print(f"• Precision: {metrics['precision']:.3f}")
    print(f"• Recall: {metrics['recall']:.3f}")
    print(f"• F1 Score: {metrics['f1']:.3f}")
    
    return metrics

def main():
    create_dataset_structure()
    
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    yaml_path = create_yaml_config()
    
    print("Training YOLOv11s model with AdamW optimizer...")
    model, training_results = train_model(yaml_path)
    
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    post_processor = ConstellationPostProcessor(
        confidence_threshold=0.25,
        nms_threshold=0.7
    )
    
    model_save_path = "models/constellation_detector_yolo11s_adamw_postprocessed.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")
    print(f"Model saved to {model_save_path}")

    
    print("Constellation detection model with post-processing training complete!")

if __name__ == "__main__":
    main()

# testing cuda installation

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")
if torch.cuda.is_available():
    print(f"GPU device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")

print(f"PyTorch version: {torch.__version__}")
print(f"Built with CUDA: {torch.version.cuda}")

PyTorch version: 2.7.0+cu118
CUDA available: True
CUDA version: 11.8
GPU device count: 1
Current device: 0
Device name: NVIDIA GeForce GTX 1070
PyTorch version: 2.7.0+cu118
Built with CUDA: 11.8
